# Inference using the BEMB model

This tutorial covers methods (e.g., `predict_proba()` and `forward()`) for post-estimation inference.

Author: Tianyu Du

Date: Aug. 8, 2022

In [1]:
import itertools
import sys
import unittest
from typing import Optional, Dict

import numpy as np
import pandas as pd
import torch
from bemb.model import BEMBFlex

# we use the dataset simulation method from unit tests.
sys.path.append('../../tests')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import simulate_choice_dataset
import torch
from bemb.model import LitBEMBFlex
from bemb.utils.run_helper import run
from torch_choice.data import ChoiceDataset


/Users/tianyudu/miniforge3/envs/ml/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Generate Simulated Datasets

In [2]:
num_users = 1500
num_items = 50
data_size = 10000
num_seeds = 32

# train, validation and test sets.
dataset_list = simulate_choice_dataset.simulate_dataset(num_users=num_users, num_items=num_items, data_size=data_size)

No `session_index` is provided, assume each choice instance is in its own session.


In [3]:
obs2prior = True
LATENT_DIM = 10  # the dimension of alpha and theta.

bemb = LitBEMBFlex(
    learning_rate=0.03,  # set the learning rate, feel free to play with different levels.
    pred_item=True,  # let the model predict item_index, don't change this one.
    num_seeds=32,  # number of Monte Carlo samples for estimating the ELBO.
    utility_formula='theta_user * alpha_item',  # the utility formula.
    num_users=num_users,
    num_items=num_items,
    num_user_obs=dataset_list[0].user_obs.shape[1],
    num_item_obs=dataset_list[0].item_obs.shape[1],
    # whether to turn on obs2prior for each parameter.
    obs2prior_dict={'theta_user': obs2prior, 'alpha_item': obs2prior},
    # the dimension of latents, since the utility is an inner product of theta and alpha, they should have
    # the same dimension.
    coef_dim_dict={'theta_user': LATENT_DIM, 'alpha_item': LATENT_DIM}
)

# use GPU if available.
if torch.cuda.is_available():
    bemb = bemb.to('cuda')
    
# use the provided run helper to train the model.
# we set batch size to be 5% of the data size, and train the model for 10 epochs.
# there would be 20*10=200 gradient update steps in total.
bemb = bemb.fit_model(dataset_list, batch_size=len(dataset_list[0]) // 20, num_epochs=10, num_workers=0)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | BEMBFlex | 33.0 K
-----------------------------------
33.0 K    Trainable params
0         Non-trainable params
33.0 K    Total params
0.132     Total estimated model params size (MB)


BEMB: utility formula parsed:
[{'coefficient': ['theta_user', 'alpha_item'], 'observable': None}]
==================== model received ====================
Bayesian EMBedding Model with U[user, item, session] = theta_user * alpha_item
Total number of parameters: 33000.
With the following coefficients:
ModuleDict(
  (theta_user): BayesianCoefficient(num_classes=1500, dimension=10, prior=N(H*X_obs(H shape=torch.Size([10, 50]), X_obs shape=50), Ix1.0))
  (alpha_item): BayesianCoefficient(num_classes=50, dimension=10, prior=N(H*X_obs(H shape=torch.Size([10, 50]), X_obs shape=50), Ix1.0))
)
[]
==================== data set received ====================
[Training dataset] ChoiceDataset(label=[], item_index=[8000], user_index=[8000], session_index=[8000], item_availability=[], user_obs=[1500, 50], item_obs=[50, 50], device=cpu)
[Validation dataset] ChoiceDataset(label=[], item_index=[1000], user_index=[1000], session_index=[1000], item_availability=[], user_obs=[1500, 50], item_obs=[50, 50], d

/Users/tianyudu/miniforge3/envs/ml/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/tianyudu/miniforge3/envs/ml/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 9: 100%|██████████| 23/23 [00:01<00:00, 22.19it/s, loss=1.75e+04, v_num=4, val_acc=0.448, val_ll=-3.00]

/Users/tianyudu/miniforge3/envs/ml/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



time taken: 11.031723976135254
==================== test performance ====================
Testing DataLoader 0: 100%|██████████| 25/25 [00:00<00:00, 293.97it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                   0.424
         test_ll            -3.054881313353777
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## The `forward()` Function

The `LitBEMBFlex` object is a class wrapping the actual model with training loops, to access the core model encompassed, we use `bemb.model`.

In [4]:
bemb.model

BEMBFlex(
  (coef_dict): ModuleDict(
    (theta_user): BayesianCoefficient(num_classes=1500, dimension=10, prior=N(H*X_obs(H shape=torch.Size([10, 50]), X_obs shape=50), Ix1.0))
    (alpha_item): BayesianCoefficient(num_classes=50, dimension=10, prior=N(H*X_obs(H shape=torch.Size([10, 50]), X_obs shape=50), Ix1.0))
  )
)

In [5]:
def forward(self, batch: ChoiceDataset,
            return_type: str,
            return_scope: str,
            deterministic: bool = True,
            sample_dict: Optional[Dict[str, torch.Tensor]] = None,
            num_seeds: Optional[int] = None
            ) -> torch.Tensor:
    """A combined method for inference with the model.

    Args:
        batch (ChoiceDataset): batch data containing choice information.
        return_type (str): either 'log_prob' or 'utility'.
            'log_prob': return the log-probability (by within-category log-softmax) for items
            'utility': return the utility value of items.
        return_scope (str): either 'item_index' or 'all_items'.
            'item_index': for each observation i, return log-prob/utility for the chosen item batch.item_index[i] only.
            'all_items': for each observation i, return log-prob/utility for all items.
        deterministic (bool, optional):
            True: expectations of parameter variational distributions are used for inference.
            False: the user needs to supply a dictionary of sampled parameters for inference.
            Defaults to True.
        sample_dict (Optional[Dict[str, torch.Tensor]], optional): sampled parameters for inference task.
            This is not needed when `deterministic` is True.
            When `deterministic` is False, the user can supply a `sample_dict`. If `sample_dict` is not provided,
            this method will create `num_seeds` samples.
            Defaults to None.
        num_seeds (Optional[int]): the number of random samples of parameters to construct. This is only required
            if `deterministic` is False (i.e., stochastic mode) and `sample_dict` is not provided.
            Defaults to None.
    Returns:
        torch.Tensor: a tensor of log-probabilities or utilities, depending on `return_type`.
            The shape of the returned tensor depends on `return_scope` and `deterministic`.
            -------------------------------------------------------------------------
            | `return_scope` | `deterministic` |         Output shape               |
            -------------------------------------------------------------------------
            |   'item_index` |      True       | (len(batch),)                      |
            -------------------------------------------------------------------------
            |   'all_items'  |      True       | (len(batch), num_items)            |
            -------------------------------------------------------------------------
            |   'item_index' |      False      | (num_seeds, len(batch))            |
            -------------------------------------------------------------------------
            |   'all_items'  |      False      | (num_seeds, len(batch), num_items) |
            -------------------------------------------------------------------------
    """
    pass

With our simple model, for the $k$-th purchasing record (observation) in the dataset, suppose `dataset.user_index[k] = ` $u(k)$ and `dataset.user_index[k] = ` $i(k)$.
Suppose there are $K$ such observations in the dataset.

The model predicts $U_{u(k) \ell} = \theta_{u(k)}^\top \alpha_\ell$ for every possible item $\ell$ including the chosen $i(k)$. This is called user $u$'s *utility* from buying item $\ell$.

### Case 1: with `pred_item == True`
In this case, one wishes to know what's the utility/probability for items.

### Case 2: with `pred_item == False`
We have a label $y_k \in \{0, 1\}$ for each observation $k$, the model can either output the (1) raw utility or (2) the predicted $\hat{P}(y_k = 1)$ defined as $\sigma(U)$, where $\sigma(x) = \frac{1}{1 + \exp(-x)}$.

**Notes**
1. the return shape does not depend on the `return_type`, please compare exact expressions to see the difference.
2. the `pred_item` variable was specified while initializing the model (see above), `return_scope` and `return_type` are supplied while calling `forward()`.

| `pred_item`| `return_scope` | `return_type`  | Output Shape | Output Tensor|
| :---: | :----: | :---: | :---: | :---: |
| `True`  | `item_index`    |  `utility`  | `(len(batch),)`| $\left[\theta_{u(1)}^\top \alpha_{i(1)}, \theta_{u(2)}^\top \alpha_{i(2)}, \dots, \theta_{u(K)}^\top \alpha_{i(K)}\right]$|
| `True`  | `item_index`    |  `log_prob`  | `(len(batch),)`| $\left[\log\left(\frac{\exp(\theta_{u(1)}^\top \alpha_{i(1)})}{\sum_{\ell \in \text{category of } i(1)} \exp(\theta_{u(1)}^\top \alpha_{\ell}) }\right), \log\left(\frac{\exp(\theta_{u(2)}^\top \alpha_{i(2)})}{\sum_{\ell \in \text{category of } i(2)} \exp(\theta_{u(2)}^\top \alpha_{\ell}) }\right), \dots, \log\left(\frac{\exp(\theta_{u(K)}^\top \alpha_{i(K)})}{\sum_{\ell \in \text{category of } i(K)} \exp(\theta_{u(K)}^\top \alpha_{\ell}) }\right)\right]$|
| `True`  | `all_items`    |  `utility`  | `(len(batch), num_items)`| $\begin{bmatrix} \theta_{u(1)}^\top \alpha_{1}, \theta_{u(1)}^\top \alpha_{2}, \dots, \theta_{u(1)}^\top \alpha_{num\_items} \\ \theta_{u(2)}^\top \alpha_{1}, \theta_{u(2)}^\top \alpha_{2}, \dots, \theta_{u(2)}^\top \alpha_{num\_items} \\ \vdots \\ \theta_{u(K)}^\top \alpha_{1}, \theta_{u(K)}^\top \alpha_{2}, \dots, \theta_{u(K)}^\top \alpha_{num\_items} \end{bmatrix}$|
| `True`  | `all_items`    |  `log_prob`  | `(len(batch), num_items)`| $\begin{bmatrix} \log\left(\frac{\exp(\theta_{u(1)}^\top \alpha_{1})}{\sum_{\ell \in \text{category of } 1} \exp(\theta_{u(1)}^\top \alpha_{\ell}) }\right), \log\left(\frac{\exp(\theta_{u(1)}^\top \alpha_{2})}{\sum_{\ell \in \text{category of } 2} \exp(\theta_{u(1)}^\top \alpha_{\ell}) }\right), \dots, \log\left(\frac{\exp(\theta_{u(1)}^\top \alpha_{num\_items})}{\sum_{\ell \in \text{category of } num\_items} \exp(\theta_{u(1)}^\top \alpha_{\ell}) }\right) \\ \log\left(\frac{\exp(\theta_{u(2)}^\top \alpha_{1})}{\sum_{\ell \in \text{category of } 1} \exp(\theta_{u(2)}^\top \alpha_{\ell}) }\right), \log\left(\frac{\exp(\theta_{u(2)}^\top \alpha_{2})}{\sum_{\ell \in \text{category of } 2} \exp(\theta_{u(2)}^\top \alpha_{\ell}) }\right), \dots, \log\left(\frac{\exp(\theta_{u(2)}^\top \alpha_{num\_items})}{\sum_{\ell \in \text{category of } num\_items} \exp(\theta_{u(2)}^\top \alpha_{\ell}) }\right) \\ \vdots \\ \log\left(\frac{\exp(\theta_{u(K)}^\top \alpha_1)}{\sum_{\ell \in \text{category of } 1} \exp(\theta_{u(K)}^\top \alpha_{\ell}) }\right), \log\left(\frac{\exp(\theta_{u(K)}^\top \alpha_2)}{\sum_{\ell \in \text{category of } 2} \exp(\theta_{u(K)}^\top \alpha_{\ell}) }\right), \dots, \log\left(\frac{\exp(\theta_{u(K)}^\top \alpha_{num\_items})}{\sum_{\ell \in \text{category of } num\_items} \exp(\theta_{u(K)}^\top \alpha_{\ell}) }\right) \end{bmatrix}$|
| `False`  | `item_index`    |  `utility`  | `(len(batch),)`| $\left[\theta_{u(1)}^\top \alpha_{i(1)}, \theta_{u(2)}^\top \alpha_{i(2)}, \dots, \theta_{u(K)}^\top \alpha_{i(K)}\right]$|
| `False`  | `item_index`    |  `log_prob`  | `(len(batch),)`| $\left[\log\sigma\left(\theta_{u(1)}^\top \alpha_{i(1)}\right), \log\sigma\left(\theta_{u(2)}^\top \alpha_{i(2)}\right), \dots, \log\sigma\left(\theta_{u(K)}^\top \alpha_{i(K)}\right)\right]$|
| `False`  | `all_items`    |  `utility`  | `(len(batch), num_items)`| $\begin{bmatrix} \theta_{u(1)}^\top \alpha_{1}, \theta_{u(1)}^\top \alpha_{2}, \dots, \theta_{u(1)}^\top \alpha_{num\_items} \\ \theta_{u(2)}^\top \alpha_{1}, \theta_{u(2)}^\top \alpha_{2}, \dots, \theta_{u(2)}^\top \alpha_{num\_items} \\ \vdots \\ \theta_{u(K)}^\top \alpha_{1}, \theta_{u(K)}^\top \alpha_{2}, \dots, \theta_{u(K)}^\top \alpha_{num\_items} \end{bmatrix}$|
| `False`  | `all_items`    |  `log_prob`  | `(len(batch), num_items)`| $\begin{bmatrix} \log\sigma\left(\theta_{u(1)}^\top \alpha_{1}\right), \log\sigma\left(\theta_{u(1)}^\top \alpha_{2}\right), \dots, \log\sigma\left(\theta_{u(1)}^\top \alpha_{num\_items}\right) \\ \log\sigma\left(\theta_{u(2)}^\top \alpha_{1}\right), \log\sigma\left(\theta_{u(2)}^\top \alpha_{2})\right), \dots, \log\sigma\left(\theta_{u(2)}^\top \alpha_{num\_items}\right) \\ \vdots \\ \log\sigma\left(\theta_{u(K)}^\top \alpha_{1}\right), \log\sigma\left(\theta_{u(K)}^\top \alpha_{2}\right), \dots, \log\sigma\left(\theta_{u(K)}^\top \alpha_{num\_items}\right) \end{bmatrix}$|

### The `deterministic` Option
By default, the `forward()` function has keyword argument `deterministic = True`. In this case, the model uses the means of fitted variational distributions of $\theta$ and $\alpha$ to compute utilities and log-probabilities.

One can specify `forward(deterministic=False, num_seeds=<XXX>)`, the model will firstly sample `num_seeds` copies of $\theta$ and $\alpha$ from their variational distributions. For each copy, the model calculated utility/log-probability as described above.

Therefore, with the same `pred_item`, `return_scope`, and `return_type`, the returned tensor has shape `(num_seeds, <the shape described in the table above>)`. For example, for a model initialized with `pred_item=False`,

* `forward(batch, return_scope='all_items', return_type='log_prob', deterministic=True)` returns shape `(len(batch), num_items)` as mentioned above.
* However, `forward(batch, return_scope='all_items', return_type='log_prob', deterministic=False, num_seeds=32)` returns shape `(32, len(batch), num_items)`.

## Syntax Sugar: the `predict_proba()` Function
The `predict_proba()` Function mimics the method with the same name in scikit-learn library.

**Note**: to avoid over-flow or under-flow issues, please use the `forward()` function, which provides log-probabilities whenever possible.

In [6]:
@torch.no_grad()
def predict_proba(self, batch: ChoiceDataset) -> torch.Tensor:
    """
    Draw prediction on a given batch of dataset.

    Args:
    batch (ChoiceDataset): the dataset to draw inference on.

    Returns:
    torch.Tensor: the predicted probabilities for each class, the behavior varies by self.pred_item.
    (1: pred_item == True) While predicting items, the return tensor has shape (len(batch), num_items), out[i, j] is the predicted probability for choosing item j AMONG ALL ITEMS IN ITS CATEGORY in observation i. Please note that since probabilities are computed from within-category normalization, hence out.sum(dim=0) can be greater than 1 if there are multiple categories.
    (2: pred_item == False) While predicting external labels for each observations, out[i, 0] is the predicted probability for label == 0 on the i-th observation, out[i, 1] is the predicted probability for label == 1 on the i-th observation. Generally, out[i, 0] + out[i, 1] = 1.0. However, this could be false if under-flowing/over-flowing issue is encountered.
    We highly recommend users to use the forward function to get the log-prob instead.
    """
    pass